# Correlation Analysis

In [73]:
import numpy as np
import pandas as pd
import anndata as ad
from scipy.sparse import csr_matrix
from scipy.sparse import issparse
import scanpy as sc
import scipy.stats
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
import seaborn as sns
print(ad.__version__)

0.11.4


In [74]:
# Load young and old data
# cell_type = "CD8-positive, alpha-beta cytotoxic T cell"
# cell_type = "erythrocyte"
cell_type = "regulatory T cell"
young_path = "subsets/{}_young_donors.h5ad".format(cell_type)
old_path = "subsets/{}_old_donors.h5ad".format(cell_type)
adata_young = ad.read_h5ad(young_path)
adata_old = ad.read_h5ad(old_path)
print(adata_young)
print(adata_old)

AnnData object with n_obs × n_vars = 175 × 603
    obs: 'reference_genome', 'gene_annotation_version', 'alignment_software', 'intronic_reads_counted', 'library_id', 'assay_ontology_term_id', 'sequenced_fragment', 'cell_number_loaded', 'institute', 'is_primary_data', 'cell_type_ontology_term_id', 'author_cell_type', 'sample_id', 'sample_preservation_method', 'tissue_ontology_term_id', 'development_stage_ontology_term_id', 'sample_collection_method', 'donor_BMI_at_collection', 'tissue_type', 'suspension_derivation_process', 'suspension_enriched_cell_types', 'cell_viability_percentage', 'suspension_uuid', 'suspension_type', 'self_reported_ethnicity_ontology_term_id', 'donor_living_at_sample_collection', 'organism_ontology_term_id', 'disease_ontology_term_id', 'sex_ontology_term_id', 'Country', 'nCount_RNA', 'nFeature_RNA', 'TCR_VDJdb', 'TCRa_V_gene', 'TCRa_D_gene', 'TCRa_J_gene', 'TCRa_C_gene', 'TCRb_V_gene', 'TCRb_D_gene', 'TCRb_J_gene', 'TCRb_C_gene', 'TCR_Clonality', 'TCR_Clone_ID', 'B

In [75]:
# Check that every gene is expressed at least once in each dataset in .X
young_check = np.sum(adata_young.X != 0, axis=0)
old_check = np.sum(adata_old.X != 0, axis=0)

# Check that every number in young_check is greater than 0
if np.all(young_check > 0):
    print("All genes expressed at least once in young dataset")

# Check that every number in old_check is greater than 0
if np.all(old_check > 0):
    print("All genes expressed at least once in old dataset")

All genes expressed at least once in young dataset
All genes expressed at least once in old dataset


In [76]:
# Get the expression matrix (cells x genes)
if issparse(adata_young.X):
    expr_matrix = adata_young.X.toarray()  # Convert sparse to dense if needed
else:
    expr_matrix = adata_young.X

# Transpose to genes x cells for correlation calculation
expr_matrix = expr_matrix.T

# Calculate correlation matrix (Pearson)
corr_matrix = np.corrcoef(expr_matrix)

# Convert to DataFrame for easier handling
gene_names = adata_young.var_names
corr_df_young = pd.DataFrame(corr_matrix, index=gene_names, columns=gene_names)

In [77]:
# # Get the expression matrix (cells x genes)
# if issparse(adata_young.X):
#     expr_matrix = adata_young.X.toarray()  # Convert sparse to dense if needed
# else:
#     expr_matrix = adata_young.X

# # Transpose to genes x cells for correlation calculation
# expr_matrix = expr_matrix.T

# # Get gene names
# gene_names = adata_young.var_names
# n_genes = len(gene_names)

# # Initialize correlation and p-value matrices
# corr_matrix = np.zeros((n_genes, n_genes))
# p_matrix = np.zeros((n_genes, n_genes))

# # Calculate correlation and p-value for each gene pair
# for i in range(n_genes):
#     for j in range(i, n_genes):  # Take advantage of symmetry
#         corr, p_val = pearsonr(expr_matrix[i], expr_matrix[j])
#         corr_matrix[i, j] = corr
#         corr_matrix[j, i] = corr  # Symmetric
#         p_matrix[i, j] = p_val
#         p_matrix[j, i] = p_val  # Symmetric

# # Convert to DataFrames
# corr_df_young = pd.DataFrame(corr_matrix, index=gene_names, columns=gene_names)
# p_value_df_young = pd.DataFrame(p_matrix, index=gene_names, columns=gene_names)

# # Print the p-value DataFrame
# print(corr_df_young)
# print(p_value_df_young)

In [78]:
# Print the diagonal of the correlation matrix and check for NaN values
print("Diagonal of the correlation matrix (should be 1):")
print(corr_df_young.values.diagonal())
print("Number of NaN values in the correlation matrix:")
print(np.isnan(corr_df_young.values).sum())

Diagonal of the correlation matrix (should be 1):
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 

In [79]:
# Print the first few rows of the correlation matrix
print("First few rows of the correlation matrix:")
print(corr_df_young.head())

# Plot the correlation matrix
# plt.figure(figsize=(10, 8))
# sns.heatmap(corr_df_young, cmap='coolwarm', center=0, square=True, cbar_kws={"shrink": .8})
# plt.title("Correlation Matrix of Gene Expression")
# plt.xlabel("Genes")
# plt.ylabel("Genes")
# plt.show()

First few rows of the correlation matrix:
                 ENSG00000116251  ENSG00000116288  ENSG00000074800  \
ENSG00000116251         1.000000         0.005771        -0.050362   
ENSG00000116288         0.005771         1.000000        -0.044648   
ENSG00000074800        -0.050362        -0.044648         1.000000   
ENSG00000028137        -0.384027         0.068643         0.230392   
ENSG00000077549        -0.379126         0.110784         0.290376   

                 ENSG00000028137  ENSG00000077549  ENSG00000127483  \
ENSG00000116251        -0.384027        -0.379126        -0.297470   
ENSG00000116288         0.068643         0.110784        -0.200846   
ENSG00000074800         0.230392         0.290376         0.111292   
ENSG00000028137         1.000000         0.268548         0.321051   
ENSG00000077549         0.268548         1.000000         0.162763   

                 ENSG00000070831  ENSG00000142676  ENSG00000117602  \
ENSG00000116251        -0.470653         0.825

In [80]:
# Find top correlated genes for a specific gene of interest
target_gene = "ENSG00000166710"
if target_gene in corr_df_young.columns:
    target_correlations = corr_df_young[target_gene].sort_values(ascending=False)
    print(f"Top genes correlated with {target_gene}:")
    print(target_correlations.head(10))
else:
    print(f"{target_gene} not found in the dataset")

Top genes correlated with ENSG00000166710:
ENSG00000166710    1.000000
ENSG00000196154    0.638056
ENSG00000142669    0.630155
ENSG00000108518    0.566622
ENSG00000188612    0.550037
ENSG00000034510    0.545408
ENSG00000008517    0.526783
ENSG00000172757    0.500740
ENSG00000154451    0.480032
ENSG00000197747    0.474969
Name: ENSG00000166710, dtype: float64


In [81]:
# Get the expression matrix for old donors
if issparse(adata_old.X):
    expr_matrix_old = adata_old.X.toarray()  # Convert sparse to dense if needed
else:
    expr_matrix_old = adata_old.X

# Transpose to genes x cells for correlation calculation
expr_matrix_old = expr_matrix_old.T

# Calculate correlation matrix (Pearson) for old donors
corr_matrix_old = np.corrcoef(expr_matrix_old)

# Convert to DataFrame for easier handling
corr_df_old = pd.DataFrame(corr_matrix_old, index=gene_names, columns=gene_names)

In [82]:
# Print the diagonal of the correlation matrix and check for NaN values
print("Diagonal of the correlation matrix (should be 1):")
print(corr_df_old.values.diagonal())
print("Number of NaN values in the correlation matrix:")
print(np.isnan(corr_df_old.values).sum())

Diagonal of the correlation matrix (should be 1):
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 

In [83]:
# Find top correlated genes for a specific gene of interest in old donors
if target_gene in corr_df_old.columns:
    target_correlations_old = corr_df_old[target_gene].sort_values(ascending=False)
    print(f"Top genes correlated with {target_gene} in old donors:")
    print(target_correlations_old.head(10))
else:
    print(f"{target_gene} not found in the old donor dataset")

Top genes correlated with ENSG00000166710 in old donors:
ENSG00000166710    1.000000
ENSG00000196154    0.708981
ENSG00000142669    0.689203
ENSG00000111348    0.651324
ENSG00000008517    0.638985
ENSG00000034510    0.624617
ENSG00000108518    0.609373
ENSG00000130429    0.598153
ENSG00000111229    0.585061
ENSG00000019582    0.581412
Name: ENSG00000166710, dtype: float64


In [84]:
# Find the differences in correlation between young and old donors
diff_corr = corr_df_young - corr_df_old
print("Difference in correlation between young and old donors:")
print(diff_corr.head())

# Assert that the diagonal of the difference correlation matrix is 0
assert np.all(diff_corr.values.diagonal() <= 1.0e-10), "Diagonal of the difference correlation matrix is not zero"

# Print the dimensions of the correlation matrices
print("Dimensions of the correlation matrix for young donors:", corr_df_young.shape)
print("Dimensions of the correlation matrix for old donors:", corr_df_old.shape)

Difference in correlation between young and old donors:
                 ENSG00000116251  ENSG00000116288  ENSG00000074800  \
ENSG00000116251     1.110223e-16     4.770000e-02     6.742630e-02   
ENSG00000116288     4.770000e-02     1.110223e-16    -2.203983e-01   
ENSG00000074800     6.742630e-02    -2.203983e-01    -1.110223e-16   
ENSG00000028137     8.337909e-02    -9.151739e-02     6.325323e-02   
ENSG00000077549    -1.722198e-02    -1.534966e-01    -1.289905e-01   

                 ENSG00000028137  ENSG00000077549  ENSG00000127483  \
ENSG00000116251         0.083379    -1.722198e-02        -0.031251   
ENSG00000116288        -0.091517    -1.534966e-01        -0.207612   
ENSG00000074800         0.063253    -1.289905e-01        -0.028151   
ENSG00000028137         0.000000    -9.469903e-02         0.099315   
ENSG00000077549        -0.094699     1.110223e-16         0.009694   

                 ENSG00000070831  ENSG00000142676  ENSG00000117602  \
ENSG00000116251        -0.278745

In [85]:
# Find the top 10 genes pairs with the largest absolute difference in correlation
top_diff_genes = diff_corr.abs().unstack().sort_values(ascending=False)

In [86]:
# Find the correlation between ENSG00000197111 and ENSG00000179094 in diff corr
gene = "ENSG00000197111"
gene2 = "ENSG00000179094"
if gene in diff_corr.columns and gene2 in diff_corr.columns:
    correlation = diff_corr.loc[gene, gene2]
    print(f"Correlation between {gene} and {gene2} in the difference correlation matrix: {correlation}")

# Find the correlation between ENSG00000197111 and ENSG00000179094 in young donors
if gene in corr_df_young.columns and gene2 in corr_df_young.columns:
    correlation_young = corr_df_young.loc[gene, gene2]
    print(f"Correlation between {gene} and {gene2} in young donors: {correlation_young}")

# Find the correlation between ENSG00000197111 and ENSG00000179094 in old donors
if gene in corr_df_old.columns and gene2 in corr_df_old.columns:
    correlation_old = corr_df_old.loc[gene, gene2]
    print(f"Correlation between {gene} and {gene2} in old donors: {correlation_old}")

In [87]:
print("Top 10 gene pairs with the largest absolute difference in correlation:")
print(top_diff_genes.head(10))

Top 10 gene pairs with the largest absolute difference in correlation:
ENSG00000197111  ENSG00000163599    0.741812
ENSG00000163599  ENSG00000197111    0.741812
ENSG00000099622  ENSG00000198695    0.702067
ENSG00000198695  ENSG00000099622    0.702067
ENSG00000124614  ENSG00000167658    0.681620
ENSG00000167658  ENSG00000124614    0.681620
ENSG00000149806  ENSG00000146278    0.655192
ENSG00000146278  ENSG00000149806    0.655192
ENSG00000090104  ENSG00000197111    0.652374
ENSG00000197111  ENSG00000090104    0.652374
dtype: float64


In [88]:
# We define a significant difference as a difference greater than 0.3
significant_diff = 0.3

# We want to create a network of genes with a significant difference in correlation between young and old donors
# So we want to make a list of all the genes and for every gene we want a list with all the genes that are significantly different
# in correlation with that gene so for example if gene A has a significant difference in correlation with gene B and gene C
# we want to add B and C to the list of gene A [A: [B, C]]
# We will use a dictionary to store the results
significant_diff_dict = {}
for gene in corr_df_young.columns:
    # Get the differences in correlation for the current gene
    diff_for_gene = diff_corr[gene]
    
    # Find genes with significant differences
    significant_genes = diff_for_gene[diff_for_gene.abs() > significant_diff].index.tolist()
    
    # Store the results in the dictionary
    significant_diff_dict[gene] = significant_genes

In [89]:
# Plot the genes with the most significant differences
# We will plot the top 10 genes with the most significant differences in the dictionary
top_genes = sorted(significant_diff_dict.items(), key=lambda x: len(x[1]), reverse=True)


In [90]:
print("Top 10 gene pairs with the most amount of difference in correlation:")
top_genes_replace_list_with_length = {}
for gene, diff_genes in top_genes[:12]:
    print(f"{gene}: {len(diff_genes)}")
    top_genes_replace_list_with_length[gene] = len(diff_genes)
# print(top_genes_replace_list_with_length)

# Plot the top 10 genes with the least significant differences in the dictionary
top_genes_replace_list_with_length = {}
print("Top 10 gene pairs with the least amount of differences in correlation:")
for gene, diff_genes in top_genes[-10:]:
    print(f"{gene}: {len(diff_genes)}")
    top_genes_replace_list_with_length[gene] = len(diff_genes)
# print(top_genes_replace_list_with_length)


# Count the total number of connections in the network
total_connections = sum(len(diff_genes) for diff_genes in significant_diff_dict.values())

# Count the total number of connections in the network for the top 120 genes
total_connections_top_120 = 0
for gene, diff_genes in top_genes[:120]:
    total_connections_top_120 += len(diff_genes)

print("Total number of connections in the network:", total_connections)
print("Total number of connections in the network for the top 12 genes:", total_connections_top_120)

Top 10 gene pairs with the most amount of difference in correlation:
ENSG00000124614: 158
ENSG00000173812: 150
ENSG00000179144: 147
ENSG00000163599: 146
ENSG00000197111: 139
ENSG00000133574: 138
ENSG00000146278: 136
ENSG00000163191: 130
ENSG00000100906: 129
ENSG00000196924: 129
ENSG00000115053: 128
ENSG00000231925: 127
Top 10 gene pairs with the least amount of differences in correlation:
ENSG00000026025: 1
ENSG00000166337: 1
ENSG00000104964: 1
ENSG00000100300: 1
ENSG00000129824: 1
ENSG00000116824: 0
ENSG00000167552: 0
ENSG00000100911: 0
ENSG00000166794: 0
ENSG00000221983: 0
Total number of connections in the network: 22236
Total number of connections in the network for the top 12 genes: 11699
